In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Irritable_bowel_syndrome_(IBS)"
cohort = "GSE20881"

# Input paths
in_trait_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)"
in_cohort_dir = "../../input/GEO/Irritable_bowel_syndrome_(IBS)/GSE20881"

# Output paths
out_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE20881.csv"
out_gene_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE20881.csv"
out_clinical_data_file = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/clinical_data/GSE20881.csv"
json_path = "../../output/preprocess/Irritable_bowel_syndrome_(IBS)/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Colon biopsies from Crohns patients and healthy controls"
!Series_summary	"The aim of this study was to investigate differential intestinal gene expression in patients with Crohn's disease (CD) and controls.  172 biopsies from CD and control subjects were studied.  Endoscopic biopsies were taken at ileocolonoscopy from five specific anatomical locations including the terminal ileum for RNA extraction."
!Series_overall_design	"Biopsies from one of four anatomic locations, from healthy controls and treated (with non-biologic standard of care) or untreated CD patients."
Sample Characteristics Dictionary:
{0: ['patient: 101', 'patient: 102', 'patient: 103', 'patient: 105', 'patient: 106', 'patient: 108', 'patient: 104', 'patient: 107', 'patient: 109', 'patient: 110', 'patient: 111', 'patient: 112', 'patient: 113', 'patient: 114', 'patient: 115', 'patient: 116', 'patient: 117', 'patient: 119', 'patient: 120', 'patient: 121', 'patient: 122', 'patient: 1

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
is_gene_available = True  # Looking at background title and summary, it's a gene expression study comparing Crohn's patients and healthy controls

# 2.1 Data Availability 
# For trait: Looking at row 58, contains disease status (healthy or Crohn's)
trait_row = 58
# For age: Can be derived from birth date in row 2
age_row = 2
# For gender: Gender information is not available in the sample characteristics
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait information to binary (0 for healthy, 1 for Crohn's disease)"""
    if isinstance(value, str) and ":" in value:
        value = value.split(":", 1)[1].strip().lower()
        if "healthy" in value:
            return 0
        elif "crohns disease" in value:
            return 1
    return None

def convert_age(value):
    """Convert birth date to age (as of the study date which appears to be 2004-2005)"""
    if isinstance(value, str) and ":" in value:
        birth_date_str = value.split(":", 1)[1].strip()
        try:
            # Handle various date formats
            if "/" in birth_date_str:
                parts = birth_date_str.split("/")
                if len(parts) == 3:
                    # Format MM/DD/YY or M/D/YY
                    month, day, year = parts
                    if len(year) == 2:
                        year = "19" + year  # Assuming 19xx
                    elif len(year) == 4:
                        pass  # Year is already in YYYY format
                    
                    # Use 2005 as reference year for age calculation (from procedure dates)
                    reference_year = 2005
                    age = reference_year - int(year)
                    return age
        except:
            pass
    return None

def convert_gender(value):
    """This function is included but not used as gender data is not available"""
    return None

# 3. Save Metadata
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # File containing the clinical data must be previously loaded as clinical_data
    clinical_features = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(clinical_features)
    print("Preview of extracted clinical features:")
    print(preview)
    
    # Save the clinical features as CSV
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_features.to_csv(out_clinical_data_file, index=False)
    print(f"Clinical features saved to {out_clinical_data_file}")


Preview of extracted clinical features:
{'GSM522094': [0.0, 42.0], 'GSM522095': [0.0, 43.0], 'GSM522096': [0.0, 40.0], 'GSM522097': [0.0, 21.0], 'GSM522098': [0.0, 21.0], 'GSM522099': [0.0, 18.0], 'GSM522100': [0.0, 72.0], 'GSM522101': [0.0, 33.0], 'GSM522102': [0.0, 945.0], 'GSM522103': [0.0, 48.0], 'GSM522104': [0.0, 51.0], 'GSM522105': [0.0, 23.0], 'GSM522106': [0.0, 24.0], 'GSM522107': [0.0, 24.0], 'GSM522108': [0.0, 69.0], 'GSM522109': [0.0, 77.0], 'GSM522110': [0.0, 49.0], 'GSM522111': [0.0, 38.0], 'GSM522112': [0.0, 42.0], 'GSM522113': [0.0, 33.0], 'GSM522114': [0.0, 51.0], 'GSM522115': [0.0, 42.0], 'GSM522116': [0.0, 40.0], 'GSM522117': [0.0, 33.0], 'GSM522118': [0.0, 21.0], 'GSM522119': [0.0, 18.0], 'GSM522120': [0.0, 945.0], 'GSM522121': [0.0, 43.0], 'GSM522122': [0.0, 48.0], 'GSM522123': [0.0, 51.0], 'GSM522124': [0.0, 23.0], 'GSM522125': [0.0, 24.0], 'GSM522126': [0.0, 69.0], 'GSM522127': [0.0, 77.0], 'GSM522128': [0.0, 49.0], 'GSM522129': [0.0, 38.0], 'GSM522130': [0.0, 40

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


Matrix table marker not found in first 100 lines


Index(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13',
       '14', '15', '16', '17', '18', '19', '20'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
import gzip
import pandas as pd
import os

# List files in the cohort directory to find the gene expression data file
files_in_dir = os.listdir(in_cohort_dir)
print(f"Files in directory: {files_in_dir}")

# Try to identify the gene expression file
gene_expr_file = None
for file in files_in_dir:
    if 'series_matrix' in file.lower():
        gene_expr_file = os.path.join(in_cohort_dir, file)
        break

if gene_expr_file:
    print(f"Using file: {gene_expr_file}")
    try:
        # Handle gzipped file correctly
        if gene_expr_file.endswith('.gz'):
            with gzip.open(gene_expr_file, 'rt') as f:
                # Find the start of the data matrix
                for line in f:
                    if line.startswith('!series_matrix_table_begin'):
                        break
                # Next line should be column headers
                header = next(f).strip()
                # Look at first few data rows to examine identifiers
                sample_ids = [next(f).split('\t')[0] for _ in range(10)]
                
            print("Sample gene identifiers:", sample_ids)
            
            # Determine if these look like gene symbols or need mapping
            are_probe_ids = any('_at' in id for id in sample_ids) or any(id.startswith('ILMN_') for id in sample_ids)
            are_numeric = all(id.isdigit() for id in sample_ids if id)
            
            requires_gene_mapping = are_probe_ids or are_numeric or not all(id[0].isalpha() for id in sample_ids if id)
        else:
            print("File is not gzipped, but expected a .gz file")
            requires_gene_mapping = True
    except Exception as e:
        print(f"Error examining gene identifiers: {e}")
        requires_gene_mapping = True
else:
    print("No gene expression file found")
    requires_gene_mapping = True

print(f"requires_gene_mapping = {requires_gene_mapping}")


Files in directory: ['GSE20881_family.soft.gz', 'GSE20881_series_matrix.txt.gz']
Using file: ../../input/GEO/Irritable_bowel_syndrome_(IBS)/GSE20881/GSE20881_series_matrix.txt.gz
Sample gene identifiers: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 2. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


Gene annotation preview:
{'ID': ['1', '2', '3', '4', '5'], 'COL': ['103', '103', '103', '103', '103'], 'ROW': [430.0, 428.0, 426.0, 424.0, 422.0], 'NAME': ['BrightCorner', 'NegativeControl', 'NM_001003689', 'NM_005503', 'NM_004672'], 'SPOT_ID': ['BrightCorner', 'NegativeControl', 'A_23_P80353', 'A_23_P158231', 'A_32_P223017'], 'CONTROL_TYPE': ['pos', 'neg', 'FALSE', 'FALSE', 'FALSE'], 'REFSEQ': [nan, nan, 'NM_001003689', 'NM_005503', 'NM_004672'], 'GB_ACC': [nan, nan, 'NM_001003689', 'NM_005503', 'NM_004672'], 'GENE': [nan, nan, 83746.0, 321.0, 9064.0], 'GENE_SYMBOL': [nan, nan, 'L3MBTL2', 'APBA2', 'MAP3K6'], 'GENE_NAME': [nan, nan, 'l(3)mbt-like 2 (Drosophila)', 'amyloid beta (A4) precursor protein-binding, family A, member 2 (X11-like)', 'mitogen-activated protein kinase kinase kinase 6'], 'UNIGENE_ID': [nan, nan, 'Hs.517641', 'Hs.525718', 'Hs.194694'], 'ENSEMBL_ID': [nan, nan, 'ENST00000216237', 'ENST00000219865', 'ENST00000357582'], 'TIGR_ID': [nan, nan, 'THC2264916', 'THC2241506',

### Step 6: Gene Identifier Mapping

In [7]:
# 1. After examining the previews from previous steps:
# - Gene expression data has numerical IDs (1, 2, 3, etc.) in its index
# - Gene annotation data has 'ID' column matching these numerical IDs
# - 'GENE_SYMBOL' column contains the gene symbols we need to map to

# 2. Extract the mapping between probe IDs and gene symbols
mapping_data = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='GENE_SYMBOL')

# 3. Convert probe-level measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, mapping_data)

# 4. Preview the gene data after mapping
print("Gene data preview after mapping:")
print(preview_df(gene_data))

# 5. Ensure the output directory exists
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)

# 6. Save the gene expression data
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Gene data preview after mapping:
{'GSM522094': [-0.18819999999999998, 0.7342, -0.5223, 0.6615, 1.3971], 'GSM522095': [-0.2644, 0.23610000000000003, 0.1128, 0.1063, 0.2396], 'GSM522096': [-0.32, 0.2738, 0.0625, 0.1618, 0.3476], 'GSM522097': [-0.2627, 0.2891, 0.1878, 0.139, 0.1318], 'GSM522098': [-0.25780000000000003, 0.3607, 0.4293, 0.0722, 0.1184], 'GSM522099': [-0.24170000000000003, 0.1795, -0.1666, 0.1234, 0.1841], 'GSM522100': [-0.1961, 0.2155, 0.2197, 0.0703, 0.0984], 'GSM522101': [-0.23249999999999998, 0.4037, 0.0456, 0.1742, 0.2205], 'GSM522102': [-0.1814, 0.2915, -0.0154, 0.0737, 0.0977], 'GSM522103': [-0.0937, 0.4138, 0.0673, 0.278, 0.5211], 'GSM522104': [-0.2962, 0.1915, 0.2145, 0.145, 0.0456], 'GSM522105': [-0.242, 0.0736, 0.1543, 0.0761, -0.0113], 'GSM522106': [-0.2652, 0.4219, 0.2232, 0.2397, 0.2875], 'GSM522107': [-0.3797, 0.0758, -0.1404, 0.1503, 0.2481], 'GSM522108': [-0.3331, 0.2781, 0.5383, 0.2188, 0.4684], 'GSM522109': [-0.3078, 0.3024, -0.1255, 0.0939, 0.0913], 'GSM5

Gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE20881.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape of gene data after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# 2. Load clinical data that was extracted in Step 2
# The clinical data is transposed (samples as columns, traits as rows)
# We need to transpose it to have samples as rows and traits as columns
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0).T
clinical_data.columns = [trait, 'Age']  # Rename columns to match expected format

# Check the structure and contents of clinical data
print("Clinical data preview (after transposing):")
print(preview_df(clinical_data))
print(f"Clinical data shape: {clinical_data.shape}")

# 3. Link the clinical and genetic data
# Transpose gene data to have samples as rows, genes as columns
gene_data_t = normalized_gene_data.T
# Join the two dataframes on their index (sample IDs)
linked_data = clinical_data.join(gene_data_t)
print(f"Linked data shape: {linked_data.shape}")

# 4. Handle missing values systematically
linked_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {linked_data.shape}")

# Debug: Check if we still have data after missing value handling
if linked_data.shape[0] == 0:
    print("WARNING: All samples were removed during missing value handling!")
    print("This may indicate a problem with data alignment or excessive missing values.")

# 5. Determine if trait and demographic features are biased
is_biased, unbiased_linked_data = judge_and_remove_biased_features(linked_data, trait)

# 6. Conduct final validation
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=True,
    is_trait_available=True,
    is_biased=is_biased,
    df=unbiased_linked_data,
    note="Dataset contains gene expression data comparing Crohn's disease patients and healthy controls."
)

# 7. Save the linked data if usable
if is_usable:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    unbiased_linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable and was not saved.")

Shape of gene data after normalization: (16774, 172)


Normalized gene expression data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/gene_data/GSE20881.csv
Clinical data preview (after transposing):
{'Irritable_bowel_syndrome_(IBS)': [0.0, 0.0, 0.0, 0.0, 0.0], 'Age': [43.0, 40.0, 21.0, 21.0, 18.0]}
Clinical data shape: (171, 2)
Linked data shape: (171, 16776)


Data shape after handling missing values: (171, 16776)
For the feature 'Irritable_bowel_syndrome_(IBS)', the least common label is '0.0' with 72 occurrences. This represents 42.11% of the dataset.
The distribution of the feature 'Irritable_bowel_syndrome_(IBS)' in this dataset is fine.

Quartiles for 'Age':
  25%: 28.0
  50% (Median): 41.0
  75%: 48.0
Min: 16.0
Max: 945.0
The distribution of the feature 'Age' in this dataset is fine.



Linked data saved to ../../output/preprocess/Irritable_bowel_syndrome_(IBS)/GSE20881.csv
